In [257]:
import pandas as pd
import numpy as np
import re

In [258]:
df_coffee_review = pd.read_csv("./web_scaping/output_data_csv/coffee_bean_cleaned.csv")
df_coffee_review.head(2)

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (paragraph)
0,Colombia Pink Bourbon,95,modcup,New Jersey,['Jersey City'],Colombia,Piendamó;Cauca Department,Light,64.0,82.0,30.00,250.0,March 2023,9.0,9.0,9.0,9.0,9.0,"Wildly tropical, fruity and deep. Passion frui...",Produced by Wilton Benitez entirely of the Pin...
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,[' Taipei'],Kenya,Kiambu County,Medium-Light,60.0,77.0,11.52,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Complex, nuanced, multi-layered. Black currant...",Produced by the Kariruki family from trees of ...


In [259]:
# number of rows before drop_duplicates
df_coffee_review["coffee name"].count()

1626

In [260]:
# drop dulipcates from the same roaster name and coffee name
df_coffee_review = df_coffee_review.drop_duplicates(subset= ["coffee name", "roaster name", "coffee origin country"], keep= "first")

In [261]:
# number of rows after drop_duplicates
df_coffee_review["coffee name"].count()

1568

### removing list format in roaster city

In [262]:
# remove list format in roaster city
roaster_city = []
for cell in df_coffee_review["roaster city"]:
    cell = cell[2:-2].strip()
    cell = re.sub("', '.", ";", cell)
    roaster_city.append(cell)

df_coffee_review["roaster city"] = roaster_city
df_coffee_review.head(2)

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (paragraph)
0,Colombia Pink Bourbon,95,modcup,New Jersey,Jersey City,Colombia,Piendamó;Cauca Department,Light,64.0,82.0,30.00,250.0,March 2023,9.0,9.0,9.0,9.0,9.0,"Wildly tropical, fruity and deep. Passion frui...",Produced by Wilton Benitez entirely of the Pin...
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,Taipei,Kenya,Kiambu County,Medium-Light,60.0,77.0,11.52,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Complex, nuanced, multi-layered. Black currant...",Produced by the Kariruki family from trees of ...


## Identifying the coffee plant species in the species (paragraph)
---

In [263]:
df_coffee_review = df_coffee_review.dropna(subset= ["species (paragraph)"])

In [264]:
np.where(df_coffee_review["species (paragraph)"].values == np.NaN)

(array([], dtype=int64),)

In [265]:
# all arabica species name
coffee_species = ["Java", "Geisha", "Sudan Rume", "Bourbon", "Typica", "Caturra", "Villa Sarchi", "Pacas", "Venecia", "Tekisic", "SL28", 
                  "K7", "KP423", "Jackson 2/1257", "Bourbon Mayaguez 71", "Bourbon Mayaguez 139", "Mibirizi", "Maragogipe", "Nyasaland",
                  "Pache", "Harrar Rwanda", "Harar", "Harraghe", "POP3303/21", "SL14", "SL34", "Mundo Novo", "Catuai", "Pacamara", "Catimor", "T8667", 
                  "T5175", "Lempira", "Costa Rica 95", "Catisic", "IHCAFE 90", "Oro Azteca", "Fronton", "Anacafe 14", "Sarchimor T5296", 
                  "Cuscatleco", "Parainema", "Limani", "Obata Rojo", "IAPAR 59", "Marsellesa", "Batian", "RAB C15", "S795", "Ruiru 11", 
                  "Centroamericano", "Nayarita", "Evaluna", "Mundo Maya", "Milenio", "Starmaya", "Casiopea", "H3", "Castillo", "Heirloom",
                  "Wush Wush", "wushwush", str(74110), str(74158), "Gesha", "Landrace", "Robusta", "Mocca","Moka", "Jember"]

In [266]:
# for loop for searching the arabica species name used in that coffee bean
species_list = []
for index, items in df_coffee_review.iterrows():
    name = items["coffee name"]
    species_para = items["species (paragraph)"]
    row = []
    for species in coffee_species:
        if species.lower() in species_para.lower() or species.lower() in name.lower():
            row.append(species)
    # replace Gesha to Geisha and Jember to S795 (they are the same species)
    row = list(map(lambda x: x.replace("Gesha", "Geisha"), row))
    row = list(map(lambda x: x.replace("Jember", "S795"), row))
    row = list(map(lambda x: x.replace("Harar", "Harrar Rwanda"), row))
    row = list(map(lambda x: x.replace("Harraghe", "Harrar Rwanda"), row))
    # drop all the dup in the list
    row = [*set(row)] # set() -> return a collection of unique objects
    row_str = ";".join(row)
    species_list.append(row_str)

In [267]:
# removing species (paragraph) column
df_coffee_review = df_coffee_review.drop(columns= "species (paragraph)")
# adding a column for the arabica species
df_coffee_review["species (variety)"] = species_list

## Seperating the coffee bean dataframe into sub-dataframe
---
- Tables: Main, species, 
- create IDs: coffee_ID, country_ID, roaster_ID, species_ID, city_ID

In [268]:
# copy df_coffee_review into full_df_coffee
full_df_coffee = df_coffee_review.copy()
full_df_coffee.head(2)

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (variety)
0,Colombia Pink Bourbon,95,modcup,New Jersey,Jersey City,Colombia,Piendamó;Cauca Department,Light,64.0,82.0,30.00,250.0,March 2023,9.0,9.0,9.0,9.0,9.0,"Wildly tropical, fruity and deep. Passion frui...",Bourbon
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,Taipei,Kenya,Kiambu County,Medium-Light,60.0,77.0,11.52,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Complex, nuanced, multi-layered. Black currant...",SL34;SL28


In [269]:
# create coffee ID (1 to 1568)
coffee_np = np.arange(1, 1568)

# adding coffee_ID column into df_coffee_review
full_df_coffee["coffee_ID"] = ["C" + str(x) for x in coffee_np]
full_df_coffee.head(2)

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),...,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (variety),coffee_ID
0,Colombia Pink Bourbon,95,modcup,New Jersey,Jersey City,Colombia,Piendamó;Cauca Department,Light,64.0,82.0,...,250.0,March 2023,9.0,9.0,9.0,9.0,9.0,"Wildly tropical, fruity and deep. Passion frui...",Bourbon,C1
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,Taipei,Kenya,Kiambu County,Medium-Light,60.0,77.0,...,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Complex, nuanced, multi-layered. Black currant...",SL34;SL28,C2


In [270]:
full_df_coffee.columns

Index(['coffee name', 'coffee rating', 'roaster name', 'roaster country/state',
       'roaster city', 'coffee origin country', 'coffee origin city',
       'roast level', 'whole bean (agtron)', 'after grinding (agtron)',
       'price', 'pricing unit (g)', 'review date', 'aroma', 'acidity', 'body',
       'flavor', 'aftertaste', 'blind ass (paragraph)', 'species (variety)',
       'coffee_ID'],
      dtype='object')

### Origin Location Table

In [271]:
# origin table
origin_df = full_df_coffee.loc[:, ["coffee_ID", "coffee origin country"]]
origin_df.head(2)

,coffee_ID,coffee origin country
0,C1,Colombia
1,C2,Kenya


In [272]:
# spliting origin country into list
origin_df["coffee origin country"] = [n.split(";") for n in origin_df["coffee origin country"]]
# expanding the table by using explode
origin_df_expand = origin_df.explode("coffee origin country")
origin_df_expand.head()

,coffee_ID,coffee origin country
0,C1,Colombia
1,C2,Kenya
2,C3,Costa Rica
3,C4,Ethiopia
4,C5,Kenya


In [273]:
# creating a list to combine all the country in origin_df and roaster_df
country_list = list(origin_df_expand["coffee origin country"].unique())

### Roaster table

In [274]:
# roaster table
roaster_grp = full_df_coffee.loc[:, ["roaster name", "roaster city"]].groupby("roaster name")
roaster_df = pd.concat([roaster_grp["roaster city"].first()], axis= "columns", sort= False).reset_index()
roaster_df["roaster name"].count()

332

In [275]:
# generate number from 1 to 333
roaster_np = np.arange(1, 333)

# adding coffee_ID column into df_coffee_review
roaster_df["roaster_ID"] = ["R" + str(x) for x in roaster_np]
roaster_df.head(2)

,roaster name,roaster city,roaster_ID
0,94 Fresh Coffee,Kaohsiung,R1
1,A.R.C.,Hong Kong,R2


In [276]:
# spliting muliple cities and putting into a list
roaster_df["roaster city"] = [n.split(";") for n in roaster_df["roaster city"]]
# expanding the table
roaster_df = roaster_df.explode("roaster city")

roaster_df["roaster city"].count()

352

#### Finding country according to city name

In [277]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(timeout = 3, user_agent="PDS") # create a geolocator object

roaster_country_list = [] # create a list to store country name

for city in roaster_df["roaster city"]:
    try:
        location = geolocator.geocode(city, language= "en") # get the location data for the city
        country = location.address.split(",")[-1].strip() # select the country name from the output
        roaster_country_list.append(country)
        print(f"{city} Done")
    except:
        roaster_country_list.append("")

roaster_df["roaster country"] = roaster_country_list

Kaohsiung Done
Hong Kong Done
Pu'Er Yunnan Province Done
Thornton Done
Milwaukee Done
Santa Rosa Beach Done
Madison Done
Hong Kong Done
Seoul Done
Fort Lauderdale Done
Douliou City Done
Salem Done
Hong Kong Done
Des Moines Done
Tainan Done
Los Angeles Done
Portland Done
Taitung Done
Arlington Done
Lee Done
Lee Done
Olympia Done
Jackson Done
Hamilton Done
Mountain View Done
Hsinchu City Done
Chicago Done
San Diego Done
Taichung Done
Taichung Done
Oakland Done
Taipei Done
Kaohsiung Done
Carbondale Done
Renton Done
Youngstown Done
Columbus Done
Taipei Done
Madison Done
Anchorage Done
Tainan Done
Taipei Done
Brooklyn Done
Manassas Done
Taipei Done
Washington Done
San Diego Done
Kaohsiung Done
Taipei Done
Santa Monica Done
Brooklyn Done
Kaohsiung Done
Cambria Done
Newtown Done
Sydney Done
Richmond Done
British Columbia Done
Taipei Done
Minneapolis Done
Annapolis Done
Taichung Done
Taipei Done
Taipei Done
Sacramento Done
San Jose Done
Montrose Done
San Luis Obispo Done
Portland Done
Kaohsiun

In [278]:
roaster_df = roaster_df[["roaster_ID", "roaster name", "roaster city", "roaster country"]]
roaster_df.head(2)

,roaster_ID,roaster name,roaster city,roaster country
0,R1,94 Fresh Coffee,Kaohsiung,Taiwan
1,R2,A.R.C.,Hong Kong,China


### Country Table

In [279]:
# counting the length of country list
print(f"Before adding roaster country: {len(country_list)}")

Before adding roaster country: 36


In [280]:
# adding roaster country that is not in the country list
for country in roaster_df["roaster country"]:
    if country not in country_list:
        country_list.append(country)
print(f"After adding roaster country: {len(country_list)}")

After adding roaster country: 46


In [281]:
# generate ID for country table
country_np = np.arange(1, 47)
country_ids = ["ctry" + str(x) for x in country_np]
len(country_ids)

46

In [282]:
# create table for country name (columns: country name and country ID)
country_df = pd.DataFrame({"country_id" : country_ids,
                           "country_name" : country_list})
country_df.head(4)

,country_id,country_name
0,ctry1,Colombia
1,ctry2,Kenya
2,ctry3,Costa Rica
3,ctry4,Ethiopia


In [283]:
# merging origin_df with the country ID and remove dup column
origin_df_merge = origin_df_expand.merge(country_df, how= "left", left_on = "coffee origin country", right_on= "country_name")\
    .drop(columns = ["coffee origin country", "country_name"])
# rename country_id column into coffee origin country ID
origin_df_cleaned = origin_df_merge.rename(columns= {"country_id" : "coffee_origin_country_id"})
origin_df_cleaned.head(2)

,coffee_ID,coffee_origin_country_id
0,C1,ctry1
1,C2,ctry2


In [284]:
# merging roaster_df with the country ID and remove dup column
roaster_df_merge = roaster_df.merge(country_df, how="left", left_on= "roaster country", right_on= "country_name")\
            .drop(columns= ["roaster country", "country_name"])
# rename columns 
roaster_df_cleaned = roaster_df_merge.rename(columns= {"roaster name" : "roaster_name",
                                                       "roaster city" : "roaster_city",
                                                       "country_id" : "roaster_country_id"})
roaster_df_cleaned.head(2)

,roaster_ID,roaster_name,roaster_city,roaster_country_id
0,R1,94 Fresh Coffee,Kaohsiung,ctry15
1,R2,A.R.C.,Hong Kong,ctry34


### Roast level table

In [285]:
roast_lv_df = full_df_coffee.loc[:,["roast level"]].groupby("roast level").first()
# there is only five different roast level -> np.range(1, 6)
roast_lv_ids = ["RL" + str(x) for x in np.arange(1, 6)]

# adding the ids into roast_lv_ids
roast_lv_df["roast_lv_id"] = roast_lv_ids
roast_lv_df = roast_lv_df.reset_index()

# rename columns
roast_lv_df_cleaned = roast_lv_df.rename(columns= {"roast level" : "roast_level"})
roast_lv_df_cleaned
 

,roast_level,roast_lv_id
0,Dark,RL1
1,Light,RL2
2,Medium,RL3
3,Medium-Dark,RL4
4,Medium-Light,RL5


### Coffee main table

In [286]:
# main table
main_df = full_df_coffee.loc[:,["coffee_ID", "coffee name", "review date", "coffee rating", "roaster name", "roast level", 
                                "whole bean (agtron)", "after grinding (agtron)", "price", "pricing unit (g)", "aroma", 
                                "acidity", "body", "flavor", "aftertaste"]]
main_df.head(2)

,coffee_ID,coffee name,review date,coffee rating,roaster name,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),aroma,acidity,body,flavor,aftertaste
0,C1,Colombia Pink Bourbon,March 2023,95,modcup,Light,64.0,82.0,30.00,250.0,9.0,9.0,9.0,9.0,9.0
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,March 2023,94,Buon Caffe,Medium-Light,60.0,77.0,11.52,226.8,9.0,9.0,9.0,9.0,8.0


In [287]:
# merging the ids for roaster_df and roast level to coffee main table
main_df_merge = main_df.merge(roast_lv_df_cleaned, how= "left", left_on= "roast level", right_on= "roast_level")
main_df_merge = main_df_merge.merge(roaster_df_cleaned[["roaster_ID", "roaster_name"]], how= "left", left_on= "roaster name", right_on= "roaster_name")

main_df_merge.head(2)

,coffee_ID,coffee name,review date,coffee rating,roaster name,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),aroma,acidity,body,flavor,aftertaste,roast_level,roast_lv_id,roaster_ID,roaster_name
0,C1,Colombia Pink Bourbon,March 2023,95,modcup,Light,64.0,82.0,30.00,250.0,9.0,9.0,9.0,9.0,9.0,Light,RL2,R331,modcup
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,March 2023,94,Buon Caffe,Medium-Light,60.0,77.0,11.52,226.8,9.0,9.0,9.0,9.0,8.0,Medium-Light,RL5,R38,Buon Caffe


In [288]:
# caluate the price per gram
main_df_merge["price_per_kg_USD"] = main_df_merge["price"]/main_df_merge["pricing unit (g)"]*1000
main_df_merge.head(2)


,coffee_ID,coffee name,review date,coffee rating,roaster name,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),aroma,acidity,body,flavor,aftertaste,roast_level,roast_lv_id,roaster_ID,roaster_name,price_per_kg_USD
0,C1,Colombia Pink Bourbon,March 2023,95,modcup,Light,64.0,82.0,30.00,250.0,9.0,9.0,9.0,9.0,9.0,Light,RL2,R331,modcup,120.000000
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,March 2023,94,Buon Caffe,Medium-Light,60.0,77.0,11.52,226.8,9.0,9.0,9.0,9.0,8.0,Medium-Light,RL5,R38,Buon Caffe,50.793651


In [289]:
# dropping the unwanted columns
main_df_drop = main_df_merge.drop(columns= ["roast level", "roast_level", "roaster name", "roaster_name", "price", "pricing unit (g)"])
main_df_drop.head(2)

,coffee_ID,coffee name,review date,coffee rating,whole bean (agtron),after grinding (agtron),aroma,acidity,body,flavor,aftertaste,roast_lv_id,roaster_ID,price_per_kg_USD
0,C1,Colombia Pink Bourbon,March 2023,95,64.0,82.0,9.0,9.0,9.0,9.0,9.0,RL2,R331,120.000000
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,March 2023,94,60.0,77.0,9.0,9.0,9.0,9.0,8.0,RL5,R38,50.793651


In [290]:
main_df_cleaned = main_df_drop.rename(columns= {"coffee name" : "coffee_name", "review date" : "review_date",
                                                 "coffee rating" : "coffee_rating", "whole bean (agtron)" : "agtron_WB",
                                                 "after grinding (agtron)" : "agtron_AG"})
main_df_cleaned.head(2)

,coffee_ID,coffee_name,review_date,coffee_rating,agtron_WB,agtron_AG,aroma,acidity,body,flavor,aftertaste,roast_lv_id,roaster_ID,price_per_kg_USD
0,C1,Colombia Pink Bourbon,March 2023,95,64.0,82.0,9.0,9.0,9.0,9.0,9.0,RL2,R331,120.000000
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,March 2023,94,60.0,77.0,9.0,9.0,9.0,9.0,8.0,RL5,R38,50.793651


### Species link table

In [291]:
# species_link table
species_df = full_df_coffee.loc[:,["coffee_ID", "species (variety)"]]
# spliting the species into list and expanding the whole table
species_df["species (variety)"] = [n.split(";") for n in species_df["species (variety)"]]
species_df = species_df.explode("species (variety)")
species_df

,coffee_ID,species (variety)
0,C1,Bourbon
1,C2,SL34
1,C2,SL28
2,C3,Geisha
3,C4,
...,...,...
1621,C1563,
1622,C1564,
1623,C1565,Geisha
1624,C1566,


### Species table

In [292]:
species_ids = ["S" + str(x) for x in np.arange(1, 60)]
len(species_ids)

59

In [293]:
# reading species csv
species_full_df = pd.read_csv("./web_scaping/output_data_csv/coffee_plant_species_cleaned.csv")
species_full_df["species_id"] = species_ids
species_full_df.head(2)

,species name,brief intro,plant stature,leaf tip colour,bean size,optial altitude low (5N_5S),optial altitude id (5-15N_5-15S),optial altitude high (15N_15S),quality potential at high altitude,yield potential,disease: coffee leaf rust,disease: coffee berry disease,disease: nematodes,nutrition requirment,planting density,species_id
0,Anacafe 14,"Very high yielding variety, with rust resistan...",Dwarf/Compact,Green,Very Large,1200,900,700,Good,High,Resistant,Susceptible,Susceptible,High,4000-5000 a/ha (using single-stem pruning),S1
1,Batian,"A tall variety that combines high yields, tole...",Tall,Green or Bronze,Very Large,1000,700,400,Very Good,High,Tolerant,Resistant,Susceptible,Medium,2000-3000 a/ha (using multiple-stem pruning),S2


In [294]:
species_full_df_drop = species_full_df.drop(columns= ["brief intro", "planting density"])
species_full_df_drop.head(2)

,species name,plant stature,leaf tip colour,bean size,optial altitude low (5N_5S),optial altitude id (5-15N_5-15S),optial altitude high (15N_15S),quality potential at high altitude,yield potential,disease: coffee leaf rust,disease: coffee berry disease,disease: nematodes,nutrition requirment,species_id
0,Anacafe 14,Dwarf/Compact,Green,Very Large,1200,900,700,Good,High,Resistant,Susceptible,Susceptible,High,S1
1,Batian,Tall,Green or Bronze,Very Large,1000,700,400,Very Good,High,Tolerant,Resistant,Susceptible,Medium,S2


In [295]:
species_full_cleaned = species_full_df_drop.rename(columns= {"species name" : "species_name", "plant stature" : "plant_stature",
                                                             "leaf tip colour" : "leaf_colour", "bean size" : "bean_size",
                                                             "optial altitude low (5N_5S)" : "altitude_low_lat",
                                                             "optial altitude id (5-15N_5-15S)" : "altitude_medium_lat",
                                                             "optial altitude high (15N_15S)" : "altitude_high_lat",
                                                             "quality potential at high altitude" : "quality_at_high_altitude",
                                                             "yield potential" : "yield_potential", "disease: coffee leaf rust" : "CLR_resistant",
                                                             "disease: coffee berry disease" : "CBD_resistant",
                                                             "disease: nematodes" : "nematodes_resistant", "nutrition requirment" : "nutrition_require"})

species_full_cleaned.head(2)

,species_name,plant_stature,leaf_colour,bean_size,altitude_low_lat,altitude_medium_lat,altitude_high_lat,quality_at_high_altitude,yield_potential,CLR_resistant,CBD_resistant,nematodes_resistant,nutrition_require,species_id
0,Anacafe 14,Dwarf/Compact,Green,Very Large,1200,900,700,Good,High,Resistant,Susceptible,Susceptible,High,S1
1,Batian,Tall,Green or Bronze,Very Large,1000,700,400,Very Good,High,Tolerant,Resistant,Susceptible,Medium,S2


In [296]:
# merging the species table with the linking table
species_df_merge = species_df.merge(species_full_df[["species_id", "species name"]], how= "left", left_on= "species (variety)", right_on= "species name")
species_df_cleaned = species_df_merge.drop(columns= ["species (variety)", "species name"])
species_df_cleaned.head(2)


,coffee_ID,species_id
0,C1,S3
1,C2,S47


#### Saving all the sub tables

In [297]:
origin_df_cleaned.to_csv("./cleaned_csv/origin_country.csv", index= False)
roast_lv_df_cleaned.to_csv("./cleaned_csv/roast_lv.csv", index= False)
roaster_df_cleaned.to_csv("./cleaned_csv/roaster.csv", index= False)
species_full_cleaned.to_csv("./cleaned_csv/species_detail.csv", index= False)
species_df_cleaned.to_csv("./cleaned_csv/species_link.csv", index= False)
main_df_cleaned.to_csv("./cleaned_csv/coffee_main.csv", index= False)